In [1]:
import os
import numpy as np
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
import torch.nn as nn
import copy
import wandb


In [ ]:
seed = 69
torch.manual_seed(seed)
np.random.seed(seed)

In [2]:
wandb.login()

wandb: Currently logged in as: johex (fdge). Use `wandb login --relogin` to force relogin


True

In [3]:
wandb.init(project='alexnet', name='Alexnet-3')

In [4]:
# load Dataset
data_path = 'data/Dataset'

In [5]:
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    #https://stackoverflow.com/questions/58151507/why-pytorch-officially-use-mean-0-485-0-456-0-406-and-std-0-229-0-224-0-2
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
dataset = datasets.ImageFolder(root=data_path, transform=data_transforms)

In [7]:
train_ds, val_ds, test_ds = random_split(dataset, [0.7, 0.1, 0.2], generator=torch.Generator().manual_seed(69))

In [8]:
train_dataloader = DataLoader(train_ds, shuffle=True, num_workers=16, batch_size=600)
test_dataloader = DataLoader(test_ds)
val_dataloader = DataLoader(val_ds, batch_size=100)

In [9]:
# alexnet model from https://github.com/pytorch/vision/blob/main/torchvision/models/alexnet.py
class AlexModel(nn.Module):
    def __init__(self, num_classes=10, dropout=0.5):
        super(AlexModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [10]:
class AlexModelv2(nn.Module):
    def __init__(self, num_classes=10, dropout=0.5):
        super(AlexModelv2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),  # +1 Conv2d layer
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [11]:
class AlexModelv3(nn.Module):
    def __init__(self, num_classes=10, dropout=0.5):
        super(AlexModelv3, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),  # Added another Conv2d layerhere (so +2 total)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [12]:
class AlexModelv4(nn.Module):
    def __init__(self, num_classes=10, dropout=0.5):
        super(AlexModelv4, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            # Removed the two 256 Conv layers here and changed to output to 256
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [13]:
class AlexModelv5(nn.Module):
    # removed one 256 conv layer
    def __init__(self, num_classes=10, dropout=0.5):
        super(AlexModelv5, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [14]:
class AlexModelv6(nn.Module):
    def __init__(self, num_classes=10, dropout=0.5):
        super(AlexModelv6, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            # 3 layers less
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(192 * 6 * 6, 4096),  # Adjusted input s
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [15]:
model = AlexModelv6(num_classes=10)
wandb.watch(model, log='all')

[]

In [16]:
# initialy we had the same SGD optimezer as alexnet used but that didnt learn anything (found out with help from https://github.com/dansuh17/alexnet-pytorch/blob/master/model.py )
# so we changed to Adam optimizer
optimizer = optim.Adam(params=model.parameters(), lr=0.0001)
wandb.config.optimizer = 'Adam'
wandb.config.learning_rate = 0.0001

In [17]:
# adjust lr on plateau
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5)
wandb.config.patience = 5

In [18]:
device = torch.device("cuda:0")
model.to(device)

num_epochs = 90
wandb.epochs = num_epochs

In [19]:
# helper function(s)
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [20]:
from tqdm import tqdm
import torch

best_acc = 0.0

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
            dataloader = train_dataloader
        else:
            model.eval()
            dataloader = val_dataloader

        running_loss = 0.0
        running_corrects = 0

        progress_bar = tqdm(dataloader, desc=f"{phase.capitalize()} Epoch {epoch+1}/{num_epochs}", leave=False)

        for inputs, labels in progress_bar:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = nn.CrossEntropyLoss()(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)


        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
        # Log metrics to wandb
        wandb.log({f"{phase}_loss": epoch_loss, f"{phase}_acc": epoch_acc})

        if phase == 'val':
            old_lr = get_lr(optimizer)
            scheduler.step(epoch_loss)
            new_lr = get_lr(optimizer)
            if new_lr < old_lr:
                print(f"Learning rate reduced from {old_lr} to {new_lr}")
                wandb.log({"learning_rate": new_lr})

        # if phase == 'val' and epoch_acc > best_acc:
        #     best_acc = epoch_acc
        # 
        #     # save the best model in models dir
        #     # path_save= f'models/model_best{epoch}.pth'
        #     # torch.save(model.state_dict(), path_save)
        # 
        #     print(f"New best model found with accuracy: {best_acc:.4f}")
            # wandb.save(f'models/model_best{epoch}.pth')

# save the last model
torch.save(model.state_dict(), 'model_final.pth')
wandb.save('model_final.pth')

Epoch 1/90


Train Loss: 2.1301 Acc: 0.2124


Val Loss: 1.9042 Acc: 0.3130
Epoch 2/90


Train Loss: 1.8278 Acc: 0.3663


Val Loss: 1.6392 Acc: 0.4470
Epoch 3/90


...

Train Loss: 0.2673 Acc: 0.9151


Val Loss: 0.2553 Acc: 0.9170
Epoch 90/90


Train Loss: 0.2666 Acc: 0.9124


Val Loss: 0.2561 Acc: 0.9140


['/media/matisse/LinuxData/pyprojects/alex/wandb/run-20240413_191424-oaiod15c/files/model_final.pth']

In [21]:
# finish run wandb
wandb.finish()


learning_rate,█▂▁
train_acc,▁▃▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇████████████████████
train_loss,█▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▄▄▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇█▇███████████████████
val_loss,█▆▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,0.0
train_acc,0.91243
train_loss,0.2666
val_acc,0.914
val_loss,0.25614
